In [ ]:
# Paso 1: Instalación de librerías
# Se instalarán las librerías necesarias de Hugging Face y Pandas.
!pip install transformers pandas torch

import pandas as pd
import re
import io
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from google.colab import drive

# PASO 1: Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# PASO 2: Definir las rutas de los archivos
# Ruta del archivo original en tu Google Drive
original_file_path = '/content/drive/MyDrive/UCM/TFM/NOTICIAS_POR_ANALIZAR.csv'

In [ ]:
# PASO 3: Leer y limpiar el contenido del archivo en memoria (sin crear un archivo limpio)
print(f"Leyendo y limpiando el contenido de '{original_file_path}' en memoria...")
cleaned_lines = []
try:
    with open(original_file_path, 'r', encoding='utf-8') as infile:
        header = next(infile)
        cleaned_lines.append(header) # Guardar el encabezado

        for line in infile:
            if line.strip().startswith('"') and ',' in line[1:]:
                # Reemplazar el formato incorrecto en memoria
                line_fixed = re.sub(r'^"(\d+),""', r'\1,"', line.strip())
                line_fixed = re.sub(r'""', r'"', line_fixed)
                cleaned_lines.append(line_fixed + '\n')
            else:
                cleaned_lines.append(line)

    # Unir las líneas limpias en una sola cadena para que Pandas pueda leerlo
    cleaned_csv_string = "".join(cleaned_lines)

    # Usar `io.StringIO` para tratar la cadena como un archivo en memoria
    cleaned_csv_file = io.StringIO(cleaned_csv_string)

    # Leer el archivo virtual en un DataFrame de Pandas
    df = pd.read_csv(cleaned_csv_file)
    print(f"DataFrame cargado y limpio en memoria con {len(df)} registros.")
    print(df.head())

except FileNotFoundError:
    print(f"Error: El archivo original '{original_file_path}' no se encontró. Verifica la ruta.")
    df = None

Leyendo y limpiando el contenido de '/content/drive/MyDrive/UCM/TFM/NOTICIAS_POR_ANALIZAR.csv' en memoria...


/tmp/ipython-input-3473739433.py:26: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(cleaned_csv_file)


DataFrame cargado y limpio en memoria con 1069449 registros.
                                     ID       FECHA  \
0  70ab9305-75e8-4cb9-9ef6-c5286c3d07f8  2024-07-08   
1  ce5fcf5b-1b24-49eb-a162-2195c738abd6  2025-08-02   
2  9294d502-a662-4674-baaa-2914dbc4f637  2025-07-29   
3  9b448a0f-aa4e-4245-bae4-aaf725bf2e92  2025-07-22   
4  d666f36b-b0c0-45fa-a13d-b06d284d763c  2025-07-24   

                                             TITULAR  \
0  Trabajadores de aeropuertos franceses convocan...   
1  How South Korea's K-beauty industry is being h...   
2  Carrefour y Alcampo, cara y cruz: la primera a...   
3  Sánchez Llibre, presidente de Foment, agradece...   
4   'Ringrose doesn't realise magnitude of his call'   

                                         URL_ARCHIVO   FUENTE IDIOMA  \
0  https://web.archive.org/web/20240709000354/htt...      ABC     es   
1  https://web.archive.org/web/20250803061923/htt...      BBC     en   
2  https://web.archive.org/web/20250727135337/htt...  E

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1069449 entries, 0 to 1069448
Data columns (total 10 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   ID                     1069449 non-null  object 
 1   FECHA                  1069449 non-null  object 
 2   TITULAR                1069449 non-null  object 
 3   URL_ARCHIVO            1069449 non-null  object 
 4   FUENTE                 1069449 non-null  object 
 5   IDIOMA                 1069449 non-null  object 
 6   SENTIMIENTO_RESULTADO  30 non-null       object 
 7   PROBABILIDAD_POSITIVO  30 non-null       float64
 8   PROBABILIDAD_NEGATIVA  30 non-null       float64
 9   PROBABILIDAD_NEUTRAL   30 non-null       float64
dtypes: float64(3), object(7)
memory usage: 81.6+ MB


In [ ]:
# PASO 4: Cargar el modelo para el análisis en español
if df is not None:
    # Este es el modelo para español
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    print("\nModelo y tokenizer para español cargados correctamente.")

    # PASO 5: Definir la función de predicción para el modelo de 5 estrellas
    def get_sentiment_es(text):
        if not isinstance(text, str) or pd.isna(text):
            return 'N/A', np.nan, np.nan, np.nan

        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        probabilities = torch.softmax(outputs.logits, dim=1)[0].cpu().numpy()

        # El modelo de nlptown mapea a 5 estrellas.
        # labels: ['1 star', '2 stars', '3 stars', '4 stars', '5 stars']
        # Mapeamos estas 5 etiquetas a 3 categorías
        predicted_class_id = np.argmax(probabilities)
        if predicted_class_id in [0, 1]:  # 1 o 2 estrellas
            sentiment = 'Negativo'
        elif predicted_class_id == 2:    # 3 estrellas
            sentiment = 'Neutral'
        elif predicted_class_id in [3, 4]:  # 4 o 5 estrellas
            sentiment = 'Positivo'
        else:
            sentiment = 'N/A'

        # Las probabilidades corresponden a las 5 estrellas
        prob_1_star = probabilities[0]
        prob_2_stars = probabilities[1]
        prob_3_stars = probabilities[2]
        prob_4_stars = probabilities[3]
        prob_5_stars = probabilities[4]

        # Calculamos las probabilidades para 3 categorías
        prob_negativa = prob_1_star + prob_2_stars
        prob_neutral = prob_3_stars
        prob_positiva = prob_4_stars + prob_5_stars

        return sentiment, prob_positiva, prob_negativa, prob_neutral

    # PASO 6: Aplicar el análisis solo a los titulares en español
    print("\nIniciando el análisis de sentimiento para titulares en español ('es')...")

    # Obtener los índices de los registros en español
    spanish_indices = df[df['IDIOMA'] == 'es'].index

    # Iterar solo sobre los registros en español
    for index in spanish_indices:
        titular = df.loc[index, 'TITULAR']
        sentiment, prob_pos, prob_neg, prob_neut = get_sentiment_es(titular)

        # Llenar las columnas de sentimiento solo para los registros en español
        df.loc[index, 'SENTIMIENTO_RESULTADO'] = sentiment
        df.loc[index, 'PROBABILIDAD_POSITIVO'] = prob_pos
        df.loc[index, 'PROBABILIDAD_NEGATIVA'] = prob_neg
        df.loc[index, 'PROBABILIDAD_NEUTRAL'] = prob_neut

    print("Análisis de sentimiento completado.")
    print("\nDataFrame con los resultados finales (primeras filas con 'es'):")
    print(df[df['IDIOMA'] == 'es'].head())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]


Modelo y tokenizer cargados correctamente.

Iniciando el análisis de sentimiento para titulares en inglés ('en')...


In [ ]:
    print(df[['FECHA', 'TITULAR', 'sentimiento_resultado', 'probabilidad_positivo', 'probabilidad_negativa', 'probabilidad_neutral']].head())

        FECHA                                            TITULAR  \
0  2025-08-05                      Un verano de opa, UCO y Greco   
1  2025-08-05  La 'gran y hermosa ley' de Trump dispara la pr...   
2  2025-08-05  Renault Group inaugura Refactory Valladolid, p...   
3  2025-08-05  Otro paso atrás en el Brexit: Reino Unido y Fr...   
4  2025-08-05  “Conocí Kit Digital y me di cuenta de lo neces...   

  sentimiento_resultado  probabilidad_positivo  probabilidad_negativa  \
0               Neutral               0.000052               0.000063   
1               Neutral               0.000135               0.001131   
2               Neutral               0.000068               0.000074   
3               Neutral               0.000236               0.000190   
4               Neutral               0.000069               0.000084   

   probabilidad_neutral  
0              0.999884  
1              0.998734  
2              0.999858  
3              0.999573  
4              0.99984

In [ ]:
   # PASO 7: Guardar el resultado final en un nuevo archivo CSV en Drive
    output_file_path = '/content/drive/MyDrive/UCM/TFM/analisis_es.csv'
    df.to_csv(output_file_path, index=False)
    print(f"\nResultados guardados en '{output_file_path}'.")


Resultados guardados en '/content/drive/MyDrive/UCM/TFM/resultado_analisis_final.csv'.


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46054 entries, 0 to 46053
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   FECHA                  46054 non-null  object 
 1   TITULAR                46054 non-null  object 
 2   URL_ARCHIVO            46054 non-null  object 
 3   FUENTE                 46054 non-null  object 
 4   IDIOMA                 46054 non-null  object 
 5   sentimiento_resultado  46054 non-null  object 
 6   probabilidad_positivo  46054 non-null  float64
 7   probabilidad_negativa  46054 non-null  float64
 8   probabilidad_neutral   46054 non-null  float64
dtypes: float64(3), object(6)
memory usage: 3.2+ MB
